<a href="https://colab.research.google.com/github/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/Chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12. 강화학습을 활용한 자연어 생성

## 12.6 강화학습을 활용한 비지도학습

* 지도학습 : 높은 정확도, 높은 비용과 시간(레이블 데이터 필요)

* 비지도학습 : 데이터 확보 시 비용과 시간 훨씬 절감 가능, 효율은 떨어진다.
* 병렬 코퍼스에 비해 쉬운 단일 언어 코퍼스가 대안이 된다.
* 소량의 병렬 코퍼스 + 다량의 단일 언어 코퍼스 = 더 나은 성능 확보 가능
* back translation, copied translation에도 NMT 성능 고도화 가능하다.

### 12.6.1 비지도학습을 활용한 NMT
* Cross-lingual Language Model Pretraining
  * 오직 단일 언어 코퍼스만을 사용해 번역기 제작
  * 언어에 상관 없이 같은 의미의 문장일 경우 인코더가 같은 값으로 임베딩하도록 훈련한다.
  * 생성적 적대 신경망(GAN) 도입
  * 인코더의 출력값이 연속적인 값이므로 GAN 적용 가능하다.

* 인코더 훈련 : 언어에 상관없이 동일한 내용의 문장은 같은 값의 벡터로 인코딩한다.
  * 판별자 : 인코딩된 문장의 언어를 맞춘다.
  * 인코더 : 판별자를 속이도록 학습한다.
  * 인코더의 출력값을 가지고 디코더를 통해 원래 문장으로 잘 돌아오도록 한다.
  * 언어에 상관없이 1개씩의 인코더와 디코더 사용
  * 다른 논문에서 제안한 사전훈련 모델 사용
* 손실 함수 : 디노이징 오토인코더, 크로스 도메인 훈련, 적대적 학습


* 디노이징 오토인코더
  * 오토인코더에게 노이즈를 섞어준 소스 문장에서 노이즈 제거하면서 입력값을 출력에서 복원할 수 있도록 훈련한다.
  * 디노이징 오토인코더(denoising autoencoder)
  $$
  \mathcal{L}_{auto}(\theta_{enc},\theta_{dec}, \mathcal{Z},\ell) = \mathbb{E}_{x \sim D_{\ell}, \hat{x} \sim d(e(C(x),\ell),\ell)}[\vartriangle(\hat{x},x)]$$
  * $\hat{x} \sim d(e(C(x),\ell),\ell$ : 입력 문장 x를 노이즈 모델 C를 통해 노이즈 노이즈 더하고, 같은 언어 $\ell$로 인코딩과 디코딩 수행
  * $\vartriangle(\hat{x},x)$ : MRT에서와 같이 원문과 복원된 문장과의 차이
  * 노이즈 모델 $C(x)$ : 임의로 문장 내 단어를 드롭하거나 순서 섞기(드롭 비율 0.1, 순서 섞는 윈도우 3)

* 크로스 도메인 훈련
  * 사전 번역을 통해 사전 훈련한 저성능 번역모델 M에서 언어($\ell_2$) 노이즈 추가되어 번역된 문장 y를 다시 언어($\ell_1$) 소스 문장으로 원상 복구하는 작업을 학습

  $$y = M(x) \\
  \mathcal{L}_{cd}(\theta_{enc},\theta_{dec}, \mathcal{Z}, \ell_1, \ell_2) = \mathbb{E}_{x \sim D_{\ell_1}, \hat{x} \sim d(e(C(y), \ell_2), \ell_1)}[\vartriangle(\hat{x},x)] $$

* 적대적 학습
  * 인코더가 언어와 상관없이 항상 같은 분포로 잠재 공간에 문장 벡터를 임베딩하는지 감시하는 판별자 추가되어 적대적 학습을 진행한다.
  * 판별자 : 잠재 변수 Z의 원래 언어를 예측하도록 훈련
  * $x_i, \ell_i$ : 같은 언어 쌍(language pair)

  $$\mathcal{L}_D(\theta_D | \theta, \mathcal{Z}) = -\mathbb{E}_{(x_i, \ell_i)}[\log p_D(\ell_i | e(x_i, \ell_i))] \\
  \mathcal{L}_{adv}(\theta_{enc},\mathcal{Z}|\theta_D) = -\mathbb{E}_{(x_i, \ell_i)}[\log p_D(\ell_j | e(x_i, \ell_i))] \\
  \text{where }j = -(i-1)$$

* 최종 목적 함수 : 3가지 목적 함수 결합하여 구한다.
  * 각 $\lambda$를 통해 손실 함수 상에서 비율 조절하여 최적의 파라미터 $\theta$ 찾는다.

  $$ \mathcal{L}(\theta_{enc},\theta_{dec}, \mathcal{Z}) = \lambda_{auto}[\mathcal{L}_{auto}(\theta_{enc},\theta_{dec},\mathcal{Z}, \ell_{src}) + \mathcal{L}_{auto}(\theta_{enc},\theta_{dec},\mathcal{Z}, \ell_{tgt})]\\  +  \lambda_{cd}[\mathcal{L}_{cd}(\theta_{enc},\theta_{dec},\mathcal{Z}, \ell_{src}, \ell_{tgt}) + \mathcal{L}_{cd}(\theta_{enc},\theta_{dec},\mathcal{Z}, \ell_{tgt},\ell_{src})]\\ + \lambda_{adv} \mathcal{L}_{adv}(\theta_{enc}, \mathcal{Z} | \theta_D)
  $$

* 본 논문에서는 오직 단일 언어 코퍼스만 존재할 때 번역기 제작 방식만을 다룬다.
  * 활용 가능성은 낮다.
  * 단일 언어 코퍼스만으로는 성능이 낮다.
  * 비용을 들여 병렬 코퍼스를 직접 구축하고, 병렬 코퍼스와 다수의 단일 언어 코퍼스 합쳐 번역기 구축할 것이기 때문이다.

## 12.7 마치며
* 강화학습 알고리즘을 사용해 자연어 생성 문제의 성능을 높인다.
  * 폴리시 그래디언트로 자연어 생성에 적용

* 폴리시 그래디언트 방법 이점
  * 자기회귀 속성으로 실제 추론 방식과 다른 teacher-forcing 훈련 방법 탈피
   * 실제 추론 방식과 같은 샘플링으로 문장 생성 능력 향상 가능
  * 더 정확한 목적 함수 훈련 가능
    * 기존의 PPL
      * 번역 품질 혹은 문장 생성 품질 정확히 반영 불가하여 BLEU 혹은 다른 방법으로 모델 성능 측정 가능
      * BLEU 미분 불가능하여 PPL의 교차 엔트로피로 신경망 훈련
    * 폴리시 그래디언트는 보상 함수에 미분할 필요가 없어서 어떤 보상 함수든지 신경망 훈련 가능
* 폴리시 그래디언트 단점
  * 많은 반복 시행 필요(샘플링 기반)
  * 훈련에 많은 시간 소요하여 비효율적인 학습 진행
  * 보상함수 : 방향 없는 스칼라값 리턴하므로 보상함수 최대화하는 방향 정확히 알 수 없다.
    * 기존의 MLE 방식 : 손실 함수를 신경망 파라미터 $\theta$에 대해 미분하여 얻은 그래디언트를 통해 손실 함수 자체 최소화아여 업데이트 하는 방식
    * 이보다 훨씬 비효율적인 학습 진행
